In [ ]:
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib

# add directory with ribasim-repository
# sys.path.append("..\\..\\..\\ribasim\\python\\ribasim")
import ribasim

# add directory with ribasim_lumping-repository
# sys.path.append("..\\..")
import ribasim_lumping

import pandas as pd
import geopandas as gpd
import numpy as np
import pyarrow as pa

import warnings

warnings.simplefilter("ignore")
pd.options.mode.chained_assignment = None

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# define network name, base dir
network_name = "tki_bommelerwaard"
base_dir = Path("..\\..\\..\\ribasim_lumping_data\\")

# directory D-HYDRO basis calculations
dhydro_basis_dir = Path(base_dir, network_name, "2_dhydro\\basis\\")

# directory D-HYDRO results calculations
dhydro_results_dir = Path(base_dir, network_name, "2_dhydro\\results\\")

# directory results
results_dir = Path(base_dir, network_name, "3_ribasim\\")

# batch file used to calculate volume D-Hydro module per GridPoint
dhydro_volume_tool_bat_file = Path(r"c:\Program Files\Deltares\D-HYDRO Suite 2023.03 1D2D\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\scripts\run_dfm_volume_tool.bat")
dhydro_volume_tool_increment = 0.1

# link to ribasim_executable
path_ribasim_executable = "..\\..\\..\\ribasim_cli\\bin\\ribasim.exe"

In [ ]:
# Create networkanalysis
network = ribasim_lumping.create_ribasim_lumping_network(
    base_dir=base_dir,
    name=network_name,
    dhydro_basis_dir=dhydro_basis_dir,
    dhydro_results_dir=dhydro_results_dir,
    results_dir=results_dir,
    path_ribasim_executable=path_ribasim_executable,
    crs=28992
)

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.add_basis_network(
    source_type="dhydro",
    set_name="winter",
    model_dir=Path(dhydro_basis_dir, "winter"),
    simulation_name="tki_vanDamvanBrakel",
    dhydro_volume_tool_bat_file=dhydro_volume_tool_bat_file, 
    dhydro_volume_tool_force=False,
    dhydro_volume_tool_increment=dhydro_volume_tool_increment
);

In [ ]:
# Add targetlevels pumps and outlets
# 

In [ ]:
# # Add simulation sets for winter, summer, etc.
# network.add_simulation_data_from_set(
#     set_name="winter",
#     simulations_dir=dhydro_dir,
#     simulations_names=["tki_bommelerwaard"],
#     simulation_output_dir="fm\\output",
#     simulations_ts=[-1],
# );

In [ ]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
# network.get_network_data()
# network.export_to_geopackage()

In [ ]:
simulation_code = "tki_vanDamvanBrakel"
set_name = 'winter'
starttime = "2020-01-01 00:00:00"
endtime = "2020-02-01 00:00:00"

In [ ]:
# Load areas (discharge units: afwaterende eenheden)
areas_dir               = Path(base_dir, network_name, "1_data", "areas")
areas_file_name         = f"praktijkpeilen_bommelerwaard.shp"
areas_id_column         = "code"
network.read_areas(
    areas_file_path=Path(areas_dir, areas_file_name), 
    areas_id_column=areas_id_column
)
cmap = matplotlib.colors.ListedColormap(np.random.rand(256, 3))
network.areas_gdf.plot(column='area_code', cmap=cmap, edgecolor='black', linewidth=0.5, alpha=0.5)

In [ ]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    orifices=True,
    structures_ids_to_include=[
        # 'DKR_009969', 'DKR_010011',
    ], 
    structures_ids_to_exclude=[
        'STW_001072B', 'STW_056492B'
    ], 
    edge_ids_to_include=[
    ],
);

In [ ]:
# specify translation split_nodes to ribasim-nodes per type
split_node_type_conversion = {
    'weir': 'TabulatedRatingCurve', 
    'uniweir': 'TabulatedRatingCurve' ,
    'pump': 'Pump', 
    'culvert': 'TabulatedRatingCurve', 
    'manual': 'ManningResistance',
    'orifice' : 'TabulatedRatingCurve',
    'edge': 'ManningResistance',
}
# specify translation for specific split_nodes to ribasim-nodes
split_node_id_conversion = {
    # 'sto_AE80770024': 'ManningResistance', 
    # 'kdu_DR80740070': 'ManningResistance',
    # # duikers voor wijk Zuidwijken
    # 'kdu_DR80940046': 'ManningResistance', 
    # 'kdu_DR80950043': 'ManningResistance', 
    # 'kdu_DR80950151': 'ManningResistance', 
}

In [ ]:
network.generate_ribasim_lumping_network(
    simulation_code=simulation_code,
    split_node_type_conversion=split_node_type_conversion,
    split_node_id_conversion=split_node_id_conversion,
);

In [ ]:
network.pumps_gdf.head()

In [ ]:
network.weirs_gdf.head()

In [ ]:
# Generate Ribasim model en exporteer naar geopackage
ribasim_model = network.generate_ribasim_model_complete(
    set_name="winter",
    use_laterals_basis_network=True,
)
ribasim_model.write(results_dir / "ribasim_model")

In [ ]:
network.laterals_data